In [32]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import datetime as dt


# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct



In [33]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///NYC_Restaurant_Inspection_Results_Clean2020.sqlite")
conn = engine.connect()

# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

# View all of the classes that automap found
inspector = inspect(engine)
inspector.get_table_names()

['NYCInspect']

In [34]:
Base.classes.keys()

['NYCInspect']

In [13]:
# checking out columns for 'NYCInspect'
columnsN = inspector.get_columns('NYCInspect')
for n in columnsN:
    print(n['name'], n['type'])

ID INTEGER
CAMIS INTEGER
DBA VARCHAR
BORO VARCHAR
BUILDING VARCHAR
STREET VARCHAR
ZIPCODE VARCHAR
PHONE VARCHAR
CUISINE DESCRIPTION VARCHAR
INSPECTION DATE VARCHAR
ACTION VARCHAR
VIOLATION CODE VARCHAR
VIOLATION DESCRIPTION VARCHAR
CRITICAL FLAG VARCHAR
SCORE INTEGER
GRADE VARCHAR
GRADE DATE VARCHAR
RECORD DATE VARCHAR
INSPECTION TYPE VARCHAR
INSPECTION TYPE GENERAL VARCHAR
Latitude FLOAT
Longitude FLOAT
Community Board VARCHAR
Council District VARCHAR
Census Tract VARCHAR
BIN VARCHAR
BBL INTEGER
NTA VARCHAR
YEAR INTEGER
ADDRESS VARCHAR


In [14]:
# Save references to each table
nyc = Base.classes.NYCInspect


# Create our session (link) from Python to the DB
session = Session(engine)

In [31]:
# nycLat = session.query(nyc.Latitude).\
# filter(nyc.GRADE != '').\
# order_by(nyc.GRADE).all()
# nycbyG

nycLat = session.query(nyc.Latitude, nyc.Longitude).distinct().filter(nyc.Latitude != 0.0).all()
nycLat

nycLat_dict = {lat: long for lat, long in nycLat}
nycLat_dict.jsonify        
    
    
    
#     return jsonify(nycLat_dict)




AttributeError: 'dict' object has no attribute 'jsonify'